In [1]:
import os

# Run in the repository base directory.
os.chdir("/home/daniel/git/cotton_flower_mot")

In [2]:
from cotton_flower_mot.pipelines.model_training.layers import CUSTOM_LAYERS
from kedro.extras.datasets.tensorflow import TensorFlowModelDataset

# Set up custom layers.
if "custom_objects" not in TensorFlowModelDataset.DEFAULT_LOAD_ARGS:
    TensorFlowModelDataset.DEFAULT_LOAD_ARGS["custom_objects"] = {}
TensorFlowModelDataset.DEFAULT_LOAD_ARGS["custom_objects"].update(
    CUSTOM_LAYERS
)

In [3]:
%reload_kedro

2024-03-05 16:45:28,384 - kedro.ipython - INFO - Resolved project path as: /home/daniel/git/cotton_flower_mot.
To set a different path, run '%reload_kedro <project_root>'
2024-03-05 16:45:28,390 - py.warnings - WARNING - /home/daniel/git/cotton_flower_mot/.venv/lib/python3.10/site-packages/kedro/framework/startup.py:98: DeprecationWarning: project_version in pyproject.toml is deprecated, use kedro_init_version instead
  warnings.warn(

2024-03-05 16:45:28,634 - kedro.io.core - WARNING - 'version' attribute removed from data set configuration since it is a reserved word and cannot be directly specified
2024-03-05 16:45:28,635 - kedro.io.core - WARNING - 'version' attribute removed from data set configuration since it is a reserved word and cannot be directly specified
2024-03-05 16:45:28,635 - kedro.io.core - WARNING - 'version' attribute removed from data set configuration since it is a reserved word and cannot be directly specified
2024-03-05 16:45:28,636 - kedro.io.core - WARNING - '

In [4]:
from kedro.runner import SequentialRunner

config_pipeline = pipelines["model_config"]
outputs = SequentialRunner().run(config_pipeline, catalog=catalog)
model_config = outputs["model_config"]

2024-03-05 16:45:30,160 - py.warnings - WARNING - /home/daniel/git/cotton_flower_mot/.venv/lib/python3.10/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":

2024-03-05 16:45:30,161 - py.warnings - WARNING - /home/daniel/git/cotton_flower_mot/.venv/lib/python3.10/site-packages/setuptools/_distutils/version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)

2024-03-05 16:45:30,205 - py.warnings - WARNING - /home/daniel/git/cotton_flower_mot/.venv/lib/python3.10/site-packages/seaborn/cm.py:1582: MatplotlibDeprecationWarning: The register_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps.register(name)`` instead.
  mpl_cm.register_cmap(_name, _cmap)

2024-03-05 16:45:30,205 - py.warnings - WARNING - /home/daniel/git/cott

2024-03-05 16:45:30.379 | DEBUG    | cotton_flower_mot.pipelines.model_config.nodes:make_model_config:23 - Creating model configuration: {'image_input_shape': [100, 100, 3], 'detection_model_input_shape': [540, 960, 3], 'rot_net_input_shape': [360, 360, 3], 'colorization_input_shape': [270, 480, 1], 'colorization_output_shape': [72, 120, 32], 'frame_input_shape': [1080, 1920, 3], 'num_appearance_features': 392, 'num_node_features': 128, 'num_edge_features': 128, 'sinkhorn_lambda': 5.0, 'num_reduction_stages': 2, 'detection_sigma': 3, 'nominal_detection_size': [0.052, 0.093], 'roi_pooling_size': 7}


2024-03-05 16:45:30,379 - kedro.io.data_catalog - INFO - Saving data to 'model_config' (MemoryDataSet)...
2024-03-05 16:45:30,380 - kedro.runner.sequential_runner - INFO - Completed 1 out of 1 tasks
2024-03-05 16:45:30,380 - kedro.runner.sequential_runner - INFO - Pipeline execution completed successfully.
2024-03-05 16:45:30,380 - kedro.io.data_catalog - INFO - Loading data from 'model_config' (MemoryDataSet)...


In [5]:
from cotton_flower_mot.pipelines.model_evaluation.inference import build_inference_model


raw_tracker_model = catalog.load("best_model")
parameters = catalog.load("parameters")

tracking_model, detection_model = build_inference_model(
    training_model=raw_tracker_model,
    config=model_config,
    detector_model_path=parameters["inference_yolo_model_path"],
    nms_iou_threshold=parameters["nms_iou_threshold"],
)


2024-03-05 16:45:32,959 - kedro.io.data_catalog - INFO - Loading data from 'best_model' (TensorFlowModelDataset)...
2024-03-05 16:45:32,986 - tensorflow - WARNING - Mixed precision compatibility check (mixed_float16): WARNING
The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


2024-03-05 16:45:32.987 | INFO     | cotton_flower_mot.pipelines.model_training.layers.pretrained_tf:reload:36 - Loading TF model from data/06_models/best_with_features_32.
2024-03-05 16:45:33.813 | DEBUG    | cotton_flower_mot.pipelines.model_training.layers.gnn:build:221 - Adding extra convolution to residual_cens_net to convert from 392 node channels to 128.


2024-03-05 16:45:33,867 - py.warnings - WARNING - /home/daniel/git/cotton_flower_mot/.venv/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(

2024-03-05 16:45:35,281 - py.warnings - WARNING - /home/daniel/git/cotton_flower_mot/.venv/lib/python3.10/site-packages/keras/src/saving/legacy/hdf5_format.py:777: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  return [np.asarray(f[weight_name]) for weight_name in weight_names]

2024-03-05 16:45:35,314 - kedro.io.data_catalog - INFO - Loading data from 'parameters' (MemoryDataSet)...


2024-03-05 16:45:35.315 | DEBUG    | cotton_flower_mot.pipelines.model_evaluation.inference:build_inference_model:135 - Building inference model...
2024-03-05 16:45:35.318 | INFO     | cotton_flower_mot.pipelines.model_training.layers.pretrained_tf:reload:36 - Loading TF model from data/06_models/best_with_features_32.
2024-03-05 16:45:35.842 | DEBUG    | cotton_flower_mot.pipelines.model_training.layers.gnn:build:221 - Adding extra convolution to residual_cens_net to convert from 392 node channels to 128.
2024-03-05 16:45:36.757 | INFO     | cotton_flower_mot.pipelines.model_training.layers.pretrained_tf:reload:36 - Loading TF model from data/06_models/best_with_features_single_image.


In [6]:
from typing import Tuple
import tensorflow as tf

def _ragged_to_normal_inputs(ragged_input: tf.RaggedTensor, name: str) -> Tuple[tf.keras.Input, tf.keras.Input, tf.RaggedTensor]:
    # Converts a ragged input into normal inputs for the flat values and row lengths.
    # Also returns the converted ragged tensor for feeding to the original input.
    _, _, num_features = ragged_input.get_shape()
    
    flat_input = tf.keras.Input(batch_size=1, shape=(None, num_features), name=f"{name}_flat", dtype=ragged_input.dtype)
    row_lengths_input = tf.keras.Input(batch_size=1, shape=(None,), name=f"{name}_row_lengths", dtype=tf.int32)
    
    # Internally, we have to convert back from the non-ragged form to the ragged form.
    converted_to_ragged = tf.RaggedTensor.from_row_lengths(flat_input[0], row_lengths_input[0])
    
    return flat_input, row_lengths_input, converted_to_ragged

In [7]:
import keras
from cotton_flower_mot.schemas import ModelInputs

# Remove ragged tensors from the input of the model.
detection_appearance, tracklet_appearance, detection_geometry, tracklet_geometry = tracking_model.inputs
detection_app_flat, detection_app_row_lengths, detection_app_ragged = _ragged_to_normal_inputs(detection_appearance, "detection_appearance")
tracklet_app_flat, tracklet_app_row_lengths, tracklet_app_ragged = _ragged_to_normal_inputs(tracklet_appearance, "tracklet_appearance")
detection_geo_flat, detection_geo_row_lengths, detection_geo_ragged = _ragged_to_normal_inputs(detection_geometry, "detection_geometry")
tracklet_geo_flat, tracklet_geo_row_lengths, tracklet_geo_ragged = _ragged_to_normal_inputs(tracklet_geometry, "tracklet_geometry")

sinkhorn, _ = tracking_model({
    ModelInputs.DETECTION_APPEARANCE.value: detection_app_ragged,
    ModelInputs.TRACKLET_APPEARANCE.value: tracklet_app_ragged,
    ModelInputs.DETECTION_GEOMETRY.value: detection_geo_ragged,
    ModelInputs.TRACKLET_GEOMETRY.value: tracklet_geo_ragged
})

# Remove ragged tensors from the outputs of the model.
sinkhorn_padded = sinkhorn.to_tensor(name="sinkhorn")
row_lengths = sinkhorn.row_lengths(name="row_lengths")
row_lengths = tf.cast(row_lengths, tf.int32)

tracking_model_no_ragged = keras.Model(inputs=[
        detection_app_flat,
        detection_app_row_lengths,
        tracklet_app_flat,
        tracklet_app_row_lengths,
        detection_geo_flat,
        detection_geo_row_lengths,
        tracklet_geo_flat,
        tracklet_geo_row_lengths
    ],
    outputs=dict(sinkhorn=sinkhorn_padded,
                 row_lengths=row_lengths)
)

In [8]:
boxes, appearance = detection_model.outputs

boxes_padded = boxes.to_tensor(name="boxes")
appearance_padded = appearance.to_tensor(name="appearance")
num_boxes = boxes.row_lengths(name="num_boxes")

detection_model_no_ragged = keras.Model(inputs=detection_model.inputs, outputs=dict(boxes=boxes_padded, appearance=appearance_padded, num_boxes=num_boxes))

In [9]:
import keras

keras.models.save_model(detection_model_no_ragged, "tf_models_temp/detection_model")
keras.models.save_model(tracking_model_no_ragged, "tf_models_temp/tracking_model")

2024-03-05 16:45:47,950 - tensorflow - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2024-03-05 16:45:51,076 - tensorflow - INFO - Assets written to: tf_models_temp/detection_model/assets
2024-03-05 16:45:51,464 - tensorflow - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2024-03-05 16:45:56,230 - tensorflow - INFO - Assets written to: tf_models_temp/tracking_model/assets
